In [ ]:

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


num_epochs = 5
num_classes = 10 
batch_size = 100 
learning_rate = 0.001 

train_dataset = torchvision.datasets.CIFAR10(root='data', # Location to store dataset
                                             train=True, # Load the training set
                                             transform=transforms.ToTensor(), # Transform images to tensor
                                             download=True) # Download if not already available

test_dataset = torchvision.datasets.CIFAR10(root='data', # Location to store dataset
                                            train=False, # Load the test set
                                            transform=transforms.ToTensor())


train_loader = torch.utils.data.DataLoader(dataset=train_dataset, # Dataset for training
                                           batch_size=batch_size, # Batch size
                                           shuffle=True) # Shuffle the data at every epoch

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, # Dataset for testing
                                          batch_size=batch_size, # Batch size
                                          shuffle=False) # Don't shuffle test data

# Define a simple CNN with two convolutional layers
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__() 
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=2), # Convolutional layer
            nn.BatchNorm2d(16), 
            nn.ReLU(), 
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.fc = nn.Linear(8*8*32, num_classes)

    def forward(self, x):
        out = self.layer1(x) 
        out = self.layer2(out) 
        out = out.reshape(out.size(0), -1) 
        out = self.fc(out) 
        return out 

model = ConvNet(num_classes).to(device)

criterion = nn.CrossEntropyLoss() # Loss for classification tasks
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) # Adam optimizer

total_step = len(train_loader) 
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader): 
        images = images.to(device) 
        labels = labels.to(device) 

        outputs = model(images)
        loss = criterion(outputs, labels) # Calculate loss

        optimizer.zero_grad() # Zero the gradient buffers (clear previous gradients)
        loss.backward() # Compute gradients through backpropagation
        optimizer.step() # Update model parameters

        # Print progress every 100 batches
        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_step}], Loss: {loss.item():.4f}')

# Testing the model
model.eval()
with torch.no_grad(): 
    correct = 0
    total = 0
    for images, labels in test_loader: 
        images = images.to(device)
        labels = labels.to(device) 
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1) 
        total += labels.size(0) 
        correct += (predicted == labels).sum().item()

print(f'Test Accuracy of the model on the 10000 test images: {100 * correct / total} %')
